In [42]:
# a partir del trabajo de https://github.com/Oprishri/Supply-Chain-Analytics-Project/blob/master/Demand_Forecasting_of_items.ipynb
# Required Libraries

import pandas as pd
import re
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Dataset
  1. Dataset is used from online retail data of a gift shop.
  2. Attributes are InvoiceNo, StockCode, Description, Quantity, InvoiceDate UnitPrice, CustomerID, Country.



In [ ]:
# UTILIZAR ESTA PARTE SI ESTAS EN GOOGLE COLAB
import sys
assert sys.version_info >= (3, 5)
import os

In [ ]:
RETAIL_PATH = "https://github.com/hcgalvan/UNSAM-Machine-Learning-on-Economics/raw/main/data/"

In [ ]:
if 'google.colab' in sys.modules:
  def load_datasets_h1(datasets_path=RETAIL_PATH):
    #retail_ol_h1 = pd.read_csv('./data/Year 2009-2010_train.csv',  encoding= 'unicode_escape')
    # retail_ol_h2 = pd.read_csv('./data/Year 2010-2011_train.csv',  encoding= 'unicode_escape')
    csv_path = os.path.join(datasets_path, "Year 2009-2010_train.csv")
    return pd.read_csv(csv_path, encoding= 'unicode_escape')

  # The function loads the data as a Pandas DataFrame instance.
  retail_ol_h1 = load_datasets_h1()
  def load_datasets_h2(datasets_path=RETAIL_PATH):
      #xlsx_path = os.path.join(datasets_path, "online_retail_II.xlsx")
      #return pd.read_excel(xlsx_path,1)
      csv_path = os.path.join(datasets_path, "Year 2010-2011_train.csv")
      return pd.read_csv(csv_path, encoding= 'unicode_escape')

  # The function loads the data as a Pandas DataFrame instance.
  retail_ol_h2 = load_datasets_h2()

In [43]:
retail_ol_h1 = pd.read_csv('./data/Year 2009-2010_train.csv',  encoding= 'unicode_escape')
retail_ol_h2 = pd.read_csv('./data/Year 2010-2011_train.csv',  encoding= 'unicode_escape')

In [44]:
frames = [retail_ol_h1, retail_ol_h2]
results = pd.concat(frames)
df = results.copy()

In [45]:
df.count()

Unnamed: 0     853896
Invoice        853896
StockCode      853896
Description    850365
Quantity       853896
InvoiceDate    853896
Price          853896
Customer ID    659429
Country        853896
dtype: int64

In [46]:
df.drop(['Unnamed: 0'], axis =1, inplace=True)

In [47]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,533533,20974,12 PENCILS SMALL TUBE SKULL,24,2010-11-17 15:24:00,0.65,18172.0,United Kingdom
1,490963,82494L,WOODEN FRAME ANTIQUE WHITE,1,2009-12-08 15:49:00,7.78,NaN,United Kingdom
2,522458,21823,PAINTED METAL HEART WITH HOLLY BELL,1,2010-09-14 15:55:00,1.45,12933.0,United Kingdom
3,534035,82599,FANNY'S REST STOPMETAL SIGN,9,2010-11-19 15:50:00,2.10,13124.0,United Kingdom
4,519029,21506,"FANCY FONT BIRTHDAY CARD,",12,2010-08-13 10:15:00,0.42,15203.0,United Kingdom
...,...,...,...,...,...,...,...,...
433523,576339,90166,PINK & WHITE ROSEBUD RING,1,2011-11-14 15:27:00,4.15,14096.0,United Kingdom
433524,559543,20727,LUNCH BAG BLACK SKULL.,1,2011-07-10 13:46:00,1.65,17126.0,United Kingdom
433525,568519,23510,MINI PLAYING CARDS GYMKHANA,20,2011-09-27 12:57:00,0.42,16607.0,United Kingdom
433526,581400,22730,ALARM CLOCK BAKELIKE IVORY,1,2011-12-08 13:36:00,3.75,14056.0,United Kingdom


In [48]:
# checking actual and unique in each attibutes.

for i in df.columns:
  print("Actual number of values",i,len(df[i]))
  print("Unique number of values",i,len(df[i].unique()))

Actual number of values Invoice 853896
Unique number of values Invoice 50850
Actual number of values StockCode 853896
Unique number of values StockCode 5214
Actual number of values Description 853896
Unique number of values Description 5622
Actual number of values Quantity 853896
Unique number of values Quantity 965
Actual number of values InvoiceDate 853896
Unique number of values InvoiceDate 45660
Actual number of values Price 853896
Unique number of values Price 2427
Actual number of values Customer ID 853896
Unique number of values Customer ID 5911
Actual number of values Country 853896
Unique number of values Country 43


In [49]:
# Checking null values in each attributes

df.isnull().sum()

Invoice             0
StockCode           0
Description      3531
Quantity            0
InvoiceDate         0
Price               0
Customer ID    194467
Country             0
dtype: int64

In [50]:
# Dropped null values because imputation in not possible in Description column and Customer ID

df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [51]:
df.count()

Invoice        659429
StockCode      659429
Description    659429
Quantity       659429
InvoiceDate    659429
Price          659429
Customer ID    659429
Country        659429
dtype: int64

# Lets create new fetures from "Description" Column.

- POS tagging of Description column for taking out features as a category.

- Fetched 'noun' as a value and made it a new column name 'Product Type'

- Also take out colours from 'Description' and made a new column name "Colour_type" .

In [52]:
df.head(659429)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,533533,20974,12 PENCILS SMALL TUBE SKULL,24,2010-11-17 15:24:00,0.65,18172.0,United Kingdom
2,522458,21823,PAINTED METAL HEART WITH HOLLY BELL,1,2010-09-14 15:55:00,1.45,12933.0,United Kingdom
3,534035,82599,FANNY'S REST STOPMETAL SIGN,9,2010-11-19 15:50:00,2.10,13124.0,United Kingdom
4,519029,21506,"FANCY FONT BIRTHDAY CARD,",12,2010-08-13 10:15:00,0.42,15203.0,United Kingdom
5,504850,21927,BLUE/CREAM STRIPE CUSHION COVER,40,2010-04-18 10:13:00,1.06,14134.0,United Kingdom
...,...,...,...,...,...,...,...,...
433522,556248,22916,HERB MARKER THYME,12,2011-06-09 16:33:00,0.65,15681.0,United Kingdom
433523,576339,90166,PINK & WHITE ROSEBUD RING,1,2011-11-14 15:27:00,4.15,14096.0,United Kingdom
433524,559543,20727,LUNCH BAG BLACK SKULL.,1,2011-07-10 13:46:00,1.65,17126.0,United Kingdom
433525,568519,23510,MINI PLAYING CARDS GYMKHANA,20,2011-09-27 12:57:00,0.42,16607.0,United Kingdom


In [58]:
dataset.iloc[:,2]

0                 12 PENCILS SMALL TUBE SKULL
2         PAINTED METAL HEART WITH HOLLY BELL
3                 FANNY'S REST STOPMETAL SIGN
4                  FANCY FONT BIRTHDAY CARD, 
5            BLUE/CREAM STRIPE CUSHION COVER 
                         ...                 
433522                      HERB MARKER THYME
433523              PINK & WHITE ROSEBUD RING
433524                LUNCH BAG  BLACK SKULL.
433525            MINI PLAYING CARDS GYMKHANA
433526             ALARM CLOCK BAKELIKE IVORY
Name: Description, Length: 659429, dtype: object

In [59]:
colours = ['red','orange', 'yellow','green', 'blue', 'indigo', 'violet', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'gray', 'black', 'white', 'cream']

stop_words = set(stopwords.words('english'))
Product_type = []
Colour_type = []
dataset=df.head(659429)
for row in dataset.iloc[:,2]:
  s=" "
  description = re.sub('[^a-zA-Z]'," ", str(row).lower()) #cleaning of text data
  wordsList = nltk.word_tokenize(description) #tokenization
  wordsList = [nltk.stem.WordNetLemmatizer().lemmatize(w, 'n') for w in wordsList if not w in stop_words] # lemmitization

  flag=False
  for w in wordsList:
    if w in colours:
      Colour_type.append(w)
      flag=True
      break
  if flag==False:
    Colour_type.append("no_color") #taking out colours from description

  tagged = nltk.pos_tag(wordsList)

  for tag in tagged:
    if tag[1]=='NN' :
      s+=tag[0] +  " "
  Product_type.append(s)


In [60]:
Colour_type

['no_color',
 'no_color',
 'no_color',
 'no_color',
 'blue',
 'no_color',
 'red',
 'no_color',
 'pink',
 'no_color',
 'no_color',
 'no_color',
 'red',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'blue',
 'no_color',
 'red',
 'no_color',
 'pink',
 'no_color',
 'no_color',
 'no_color',
 'white',
 'no_color',
 'red',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'purple',
 'white',
 'no_color',
 'silver',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'yellow',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'silver',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'white',
 'no_color',
 'no_color',
 'no_color',
 'blue',
 'no_color',
 'no_color',
 'no_color',
 'no_color',
 'green',
 'no_color',
 'no_color',
 'no_co

In [61]:
# Adding new columns in dataframe 

dataset['Product Type']=Product_type
dataset['Colour_type']=Colour_type
dataset.head()

<ipython-input-61-df665c7adc5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Product Type']=Product_type
<ipython-input-61-df665c7adc5e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Colour_type']=Colour_type


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Product Type,Colour_type
0,533533,20974,12 PENCILS SMALL TUBE SKULL,24,2010-11-17 15:24:00,0.65,18172.0,United Kingdom,pencil tube skull,no_color
2,522458,21823,PAINTED METAL HEART WITH HOLLY BELL,1,2010-09-14 15:55:00,1.45,12933.0,United Kingdom,heart,no_color
3,534035,82599,FANNY'S REST STOPMETAL SIGN,9,2010-11-19 15:50:00,2.10,13124.0,United Kingdom,rest stopmetal sign,no_color
4,519029,21506,"FANCY FONT BIRTHDAY CARD,",12,2010-08-13 10:15:00,0.42,15203.0,United Kingdom,font birthday card,no_color
5,504850,21927,BLUE/CREAM STRIPE CUSHION COVER,40,2010-04-18 10:13:00,1.06,14134.0,United Kingdom,cream stripe cushion cover,blue


In [62]:
# Dropped "Description"and "InvoiceDate" columns

X = dataset.drop(["Description","InvoiceDate"],axis=1)

In [63]:
X.head()

,Invoice,StockCode,Quantity,Price,Customer ID,Country,Product Type,Colour_type
0,533533,20974,24,0.65,18172.0,United Kingdom,pencil tube skull,no_color
2,522458,21823,1,1.45,12933.0,United Kingdom,heart,no_color
3,534035,82599,9,2.10,13124.0,United Kingdom,rest stopmetal sign,no_color
4,519029,21506,12,0.42,15203.0,United Kingdom,font birthday card,no_color
5,504850,21927,40,1.06,14134.0,United Kingdom,cream stripe cushion cover,blue


In [29]:
X.head()

,Unnamed: 0,Invoice,StockCode,Quantity,Price,Customer ID,Country,Product Type,Colour_type
0,468332,533533,20974,24,0.65,18172.0,United Kingdom,pencil tube skull,no_color
2,339143,522458,21823,1,1.45,12933.0,United Kingdom,heart,no_color
3,474021,534035,82599,9,2.10,13124.0,United Kingdom,rest stopmetal sign,no_color
4,304210,519029,21506,12,0.42,15203.0,United Kingdom,font birthday card,no_color
5,161441,504850,21927,40,1.06,14134.0,United Kingdom,cream stripe cushion cover,blue


# Created a new feature name "Revenue"

  - revenue = UnitPrice * Quantity

In [64]:
X['Revenue'] = X['Price'] * X['Quantity']
X.head()

,Invoice,StockCode,Quantity,Price,Customer ID,Country,Product Type,Colour_type,Revenue
0,533533,20974,24,0.65,18172.0,United Kingdom,pencil tube skull,no_color,15.60
2,522458,21823,1,1.45,12933.0,United Kingdom,heart,no_color,1.45
3,534035,82599,9,2.10,13124.0,United Kingdom,rest stopmetal sign,no_color,18.90
4,519029,21506,12,0.42,15203.0,United Kingdom,font birthday card,no_color,5.04
5,504850,21927,40,1.06,14134.0,United Kingdom,cream stripe cushion cover,blue,42.40


In [65]:
# Label encoding of categorical features

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

for col in ["Invoice", "StockCode", "Customer ID","Country", "Product Type","Colour_type"]:
  X[col] = label_encoder.fit_transform(X[col])

In [66]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659429 entries, 0 to 433526
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Invoice       659429 non-null  int32  
 1   StockCode     659429 non-null  int32  
 2   Quantity      659429 non-null  int64  
 3   Price         659429 non-null  float64
 4   Customer ID   659429 non-null  int64  
 5   Country       659429 non-null  int32  
 6   Product Type  659429 non-null  int32  
 7   Colour_type   659429 non-null  int32  
 8   Revenue       659429 non-null  float64
dtypes: float64(2), int32(5), int64(2)
memory usage: 37.7 MB


In [34]:
!pip install kmodes 

Defaulting to user installation because normal site-packages is not writeable


In [67]:
# Changed the data type of attributes

X = X.astype('category')
X.iloc[:, 2] = X.iloc[:, 2].astype(float)
X.iloc[:, 3] = X.iloc[:, 3].astype(float)
X.iloc[:, 8] = X.iloc[:, 8].astype(float)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659429 entries, 0 to 433526
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Invoice       659429 non-null  category
 1   StockCode     659429 non-null  category
 2   Quantity      659429 non-null  float64 
 3   Price         659429 non-null  float64 
 4   Customer ID   659429 non-null  category
 5   Country       659429 non-null  category
 6   Product Type  659429 non-null  category
 7   Colour_type   659429 non-null  category
 8   Revenue       659429 non-null  float64 
dtypes: category(6), float64(3)
memory usage: 29.5 MB


In [68]:
# Train test split of new dataframe X

from sklearn.model_selection import train_test_split
train, test = train_test_split(X, train_size=0.8, random_state = 0)

# Cluster the similar items for new attribute cluster number, for this K-prototype clustering is used.

In [69]:
# Chequeo del valor optimo de 'K' // demanda varios minutos de ejecucion para encontrar el optimo

import matplotlib.pyplot as plt
from kmodes.kprototypes import KPrototypes

cost = []
for num_clusters in list(range(2,15)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao')
    kproto.fit_predict(train, categorical=[0,1,4,5,6,7])
    cost.append(kproto.cost_)
    labels=kproto.labels_
plt.plot(cost)

KeyboardInterrupt: 

In [1]:
# Se genera un nuevo numero de cluster del atributo

kproto = KPrototypes(n_clusters=3, init='Cao')
kproto.fit_predict(train, categorical=[0,1,4,5,6])
print(kproto.cost_)
labels=kproto.labels_

NameError: name 'KPrototypes' is not defined

In [71]:
# Agrego nuevo atributo

train["Cluster number"]=labels
train

<ipython-input-71-e92d93fb4fb5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Cluster number"]=labels


,Invoice,StockCode,Quantity,Price,Customer ID,Country,Product Type,Colour_type,Revenue,Cluster number
265398,29120,247,10.0,1.65,3586,38,2220,12,16.50,2
345179,29038,2489,12.0,1.25,1631,38,233,8,15.00,2
178929,33343,1729,1.0,1.65,2308,38,2987,8,1.65,2
208951,9257,1732,12.0,1.65,75,34,2985,8,19.80,2
120347,32492,1800,2.0,8.50,1517,38,2112,12,17.00,2
...,...,...,...,...,...,...,...,...,...,...
34461,33622,2360,10.0,0.85,857,38,670,8,8.50,2
191656,7326,2933,2.0,2.55,2149,38,2418,10,5.10,2
148322,2822,1387,2.0,9.95,1622,38,3168,8,19.90,2
135733,26349,2466,1.0,2.95,5466,38,1889,14,2.95,2


In [72]:
# Ahora agrego "InvoiceDate" en dataframe

nuevodDf = train.merge(pd.DataFrame(dataset["InvoiceDate"]), left_index=True, right_index=True)
nuevodDf

,Invoice,StockCode,Quantity,Price,Customer ID,Country,Product Type,Colour_type,Revenue,Cluster number,InvoiceDate
0,16914,414,24.0,0.65,5795,38,2728,8,15.60,2,2010-11-17 15:24:00
2,12403,1110,1.0,1.45,584,38,1776,8,1.45,2,2010-09-14 15:55:00
3,17114,3374,9.0,2.10,773,38,3141,8,18.90,2,2010-11-19 15:50:00
3,17114,3374,9.0,2.10,773,38,3141,8,18.90,2,2011-11-13 15:19:00
3,33942,1331,1.0,9.95,3052,38,308,8,9.95,2,2010-11-19 15:50:00
...,...,...,...,...,...,...,...,...,...,...,...
433522,25822,2084,12.0,0.65,3319,38,1915,8,7.80,2,2011-06-09 16:33:00
433523,34047,4490,1.0,4.15,1741,38,3281,10,4.15,2,2011-11-14 15:27:00
433524,27089,249,1.0,1.65,4756,38,2221,1,1.65,2,2011-07-10 13:46:00
433525,30741,2642,20.0,0.42,4240,38,584,8,8.40,2,2011-09-27 12:57:00


# Ingeniería de Feature en la columna "InvoiceDate"

In [76]:
from datetime import datetime

year = []
month = []
day = []
dayofweek = []
for row in nuevodDf["InvoiceDate"]:
  dt=datetime.strptime(row, '%Y/%m/%d %H:%M')
  year.append(dt.year)
  month.append(dt.month)
  day.append(dt.day)
  dayofweek.append(dt.strftime("%w"))

nuevodDf['Year']=year
nuevodDf["Month"]=month
nuevodDf["Day"]=day
nuevodDf['DayOfWeek']=dayofweek
nuevodDf.drop(['InvoiceDate'],axis=1, inplace=True)
nuevodDf.head()

ValueError: time data '2010-11-17 15:24:00' does not match format '%Y/%m/%d %H:%M'

In [ ]:
nuevoDf.to_csv('./data/nuevoDf.csv')

In [16]:
nuevodDf = pd.read_csv('./data/nuevodDf.csv')
nuevodDf.drop(['Unnamed: 0'], axis =1, inplace=True)
nuevodDf.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product Type,Colour_type,Revenue,Cluster number,Year,Month,Day,DayOfWeek
0,1018,2277,6.0,4.25,997,25,0,7,4.25,2,2010,12,14,2
1,888,1611,1.0,8.50,1228,25,2151,3,8.50,2,2010,12,12,0
2,2254,2107,1.0,3.75,1363,25,0,7,3.75,2,2011,1,27,4
3,1749,2189,16.0,0.65,686,25,903,7,0.65,2,2011,1,12,3
4,1607,574,1.0,4.95,961,25,1267,7,4.95,2,2011,1,9,0


# Classification of test data into number of clusters

- Cluster numbers were treated as a target variable as the objective
was to match the records from the validation and testing sets with the clusters from the training
set.


In [ ]:
# Splitting of nuevoDF dataframe into train and validation

from sklearn.model_selection import train_test_split
train_, val_= train_test_split(nuevodDf, train_size = 0.8, random_state = 0)

In [ ]:
train_y=train_["Cluster number"]
train_x=train_.drop(['Cluster number'],axis=1,inplace=False)

val_y=val_["Cluster number"]
val_x=val_.drop(['Cluster number'],axis=1,inplace=False)

For classification
  - Linear SVC is used. and it is giving best result among other machine leaning algorithms.

In [ ]:
from sklearn.svm import LinearSVC 

In [1]:
model1 = LinearSVC()
model1.fit(train_x,train_y)

In [ ]:
# validating on test data

pred_y = model1.predict(val_x)

In [2]:
# Preformance evaluation

from sklearn.metrics import accuracy_score
accuracy_score(val_y,pred_y)

In [ ]:
# Adding "InvoiceDate" in test data

test_Df = test.merge(pd.DataFrame(dataset["InvoiceDate"]), left_index=True, right_index=True)
test_Df

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product Type,Colour_type,Revenue,InvoiceDate
17873,617,729,1.0,1.25,1158,25,2158,7,1.25,12/8/2010 12:46
29644,985,1181,10.0,1.95,225,25,71,7,1.95,12/14/2010 12:22
73585,2238,1608,1.0,7.95,551,25,1793,5,7.95,1/27/2011 12:40
39495,1339,197,1.0,2.55,726,25,917,7,2.55,12/20/2010 16:08
69859,2106,121,2.0,3.25,409,25,1763,11,3.25,1/24/2011 16:05
...,...,...,...,...,...,...,...,...,...,...
38320,1300,1444,3.0,1.25,1099,25,2213,1,1.25,12/20/2010 11:08
44202,1447,2035,2.0,2.10,692,25,1254,7,2.10,1/5/2011 11:41
75671,2317,216,4.0,1.95,216,25,197,7,1.95,1/30/2011 13:48
58928,1855,1031,3.0,2.10,1225,25,1084,13,2.10,1/16/2011 16:23


In [ ]:
# Feature engineering of "InvoiceDate" column

from datetime import datetime

year = []
month = []
day = []
dayofweek = []
for row in test_Df["InvoiceDate"]:
  dt=datetime.strptime(row, '%m/%d/%Y %H:%M')
  year.append(dt.year)
  month.append(dt.month)
  day.append(dt.day)
  dayofweek.append(dt.strftime("%w"))

test_Df['Year']=year
test_Df["Month"]=month
test_Df["Day"]=day
test_Df['DayOfWeek']=dayofweek
test_Df.drop(['InvoiceDate'],axis=1, inplace=True)

test_Df=test_Df.reset_index()
test_Df.drop(["index"],axis=1, inplace = True)
test_Df.head()

test_Df.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product Type,Colour_type,Revenue,Year,Month,Day,DayOfWeek
17873,617,729,1.0,1.25,1158,25,2158,7,1.25,2010,12,8,3
29644,985,1181,10.0,1.95,225,25,71,7,1.95,2010,12,14,2
73585,2238,1608,1.0,7.95,551,25,1793,5,7.95,2011,1,27,4
39495,1339,197,1.0,2.55,726,25,917,7,2.55,2010,12,20,1
69859,2106,121,2.0,3.25,409,25,1763,11,3.25,2011,1,24,1


In [ ]:
# Prediction of cluster_number on test data

test_Df['Cluster number'] = model1.predict(test_Df)
test_Df 

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Product Type,Colour_type,Revenue,Year,Month,Day,DayOfWeek,Cluster number
0,617,729,1.0,1.25,1158,25,2158,7,1.25,2010,12,8,3,2
1,985,1181,10.0,1.95,225,25,71,7,1.95,2010,12,14,2,2
2,2238,1608,1.0,7.95,551,25,1793,5,7.95,2011,1,27,4,2
3,1339,197,1.0,2.55,726,25,917,7,2.55,2010,12,20,1,2
4,2106,121,2.0,3.25,409,25,1763,11,3.25,2011,1,24,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1300,1444,3.0,1.25,1099,25,2213,1,1.25,2010,12,20,1,2
9996,1447,2035,2.0,2.10,692,25,1254,7,2.10,2011,1,5,3,2
9997,2317,216,4.0,1.95,216,25,197,7,1.95,2011,1,30,0,2
9998,1855,1031,3.0,2.10,1225,25,1084,13,2.10,2011,1,16,0,2


# Prediction of "Quantity" column for the products demands

In [ ]:
train_y=train_["Quantity"].astype('int')
train_x=train_.drop(['Quantity'], axis=1,inplace=False)

test_Df_y=test_Df["Quantity"].astype('int')
test_Df_x=test_Df.drop(['Quantity'],axis=1,inplace=False)

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0,
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                                                   n_jobs=None,)
clf.fit(train_x, train_y)

In [ ]:
prediction_test = clf.predict(test_Df_x)
prediction_test

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_Df_y, prediction_test, average='micro')

In [1]:
accuracy_score(test_Df_y, prediction_test)

In [ ]:
# Hyperparameter tuning of RAndom forest algorithm

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
import numpy as np


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)



# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_x, train_y)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 166.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 310.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [2]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_x, train_y)
knn=neigh.predict(test_Df_x)
print(accuracy_score(test_Df_y, knn))

In [3]:
# SVC with kernel 

from sklearn import svm
from sklearn.svm import SVC

rbf_svc = svm.SVC(kernel='rbf')
rbf_svc.fit(train_x, train_y)

rbf=rbf_svc.predict(test_Df_x)
accuracy_score(test_Df_y, rbf)


In [4]:
# AdaBoost

 from sklearn.ensemble import AdaBoostClassifier

ad = AdaBoostClassifier(n_estimators=100, random_state=0)
ad.fit(train_x, train_y)
adb=ad.predict(test_Df_x)
print(accuracy_score(test_Df_y, adb))

In [17]:
# logistic

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class='ovr')
lr.fit(train_x, train_y)
lrc=lr.predict(test_Df_x)
print(accuracy_score(test_Df_y, lrc))

In [5]:
# Naive base Classifier

from sklearn.naive_bayes import GaussianNB

lr = GaussianNB()
lr.fit(train_x, train_y)
lrc=lr.predict(test_Df_x)
print(accuracy_score(test_Df_y, lrc))

In [6]:
# Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier 

dtree_model = DecisionTreeClassifier().fit(train_x, train_y)
dtree_predictions = dtree_model.predict(test_Df_x)
accuracy_score(test_Df_y, dtree_predictions)

In [ ]:
# GradientBoostingClassifier

from sklearn.ensemble import GradientBoostingClassifier

gb=GradientBoostingClassifier()
gb.fit(train_x, train_y)
gbc=lr.predict(test_Df_x)
print(accuracy_score(test_Df_y, gbc))